### LLama-cpp-python ile kullanımı için Türkçe kaynak

llama.cpp, Meta'nın LLaMA ve diğer desteklediği modellerini CPU ve düşük kaynaklı donanımlarda çalıştırmak amacıyla yazılmış, C/C++ tabanlı, oldukça hızlı ve hafif bir inference (çıkarım) yapmakta.

Aşağıda hem llama.cpp'in hem de llama-cpp-python'ın ne olduğu, nasıl kullanıldığı ve hangi özellikleri desteklediği üzerine detaylı bir anlatım yapacağım.

- LLaMA ve diğer desteklediği modellerini yerel olarak çalıştırabilir.
- CUDA, Metal (Apple), OpenBLAS, AVX2 gibi hızlandırmalarla performanslı inference yapabilir,
- API server olarak çalıştırıp OpenAI API uyumlu hale getirebilir,
- AVX, AVX2, AVX512, NEON gibi SIMD optimizasyonları
- GGUF formatındaki modelleri destekler 
- Yerel Çalıştırma: Bulut bağımlılığı olmadan kendi donanımınızda çalıştırma
- Hafifletilmiş Modeller: GGUF formatıyla quantize edilmiş (küçültülmüş) modeller

In [3]:
import llama_cpp
llama_cpp.__version__

'0.3.8'

In [51]:
import torch
print(f"GPU mu kullanılıyor: {torch.cuda.is_available()}")
print(f"GPU Adı: {torch.cuda.get_device_name(0)}")

GPU mu kullanılıyor: True
GPU Adı: NVIDIA RTX A5000


### GGUF Model Dosyasını İndirmek

Bunun için birkaç yöntem göstereceğim.

1) HF üzerinden direkt locale indrimek. Donanım durumunuza göre İstediğiniz quantize modeli indirebilirsiniz.
2) Hf ile aşağıdaki kod ile indirebilirsiniz.

Quantize seviyesi (Q4_K_M, Q5_K_S vb.) model kalitesi ve boyutu arasında trade-off sağlar.
Bazı modeller (özellikle orijinal Meta modelleri) için erişim izni gerekebilir.

In [2]:
from huggingface_hub import hf_hub_download

model_name = "unsloth/gemma-3-12b-it-GGUF"
model_file = "gemma-3-12b-it-Q8_0.gguf"

model_path = hf_hub_download(
    repo_id=model_name,
    filename=model_file,
    local_dir="./models"
)

/root/course/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


### Şimdi llama_cpp kullanabilriz.

In [5]:
from llama_cpp import Llama

llm = Llama(
    model_path= "./models/gemma-3-12b-it-Q8_0.gguf", #locale indirdiğimiz modeli alıyoruz.
    n_gpu_layers=-1,
    seed = 1337,
    n_ctx=2048,
    verbose = False
)

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [6]:
response = llm("Q: Selam A: ",
           max_tokens=32,
           stop=["Q:", "\n"],
            echo = True,
)

In [ ]:
response

{'id': 'cmpl-fdd08b36-9414-462e-84a3-94ad889372a3',
 'object': 'text_completion',
 'created': 1745514201,
 'model': './models/llama-2-7b.Q4_K_M.gguf',
 'choices': [{'text': 'Q: Selam A: 01. nobody can tell me why the following code does not work?',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 8, 'completion_tokens': 16, 'total_tokens': 24}}

In [ ]:
response['choices'][0]['text']

'Q: Selam A: 01. nobody can tell me why the following code does not work?'

### HF'den pulling model çekme işlemi
Llama.from_pretrained ile modeli çekiyor

In [5]:
from llama_cpp import Llama
llm_12b = Llama.from_pretrained(
    repo_id = "unsloth/gemma-3-12b-it-GGUF",
    filename="gemma-3-12b-it-Q4_K_M.gguf",
    verbose = False
)

/root/course/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [9]:
resp = llm_12b("Q: Selam nasılsın A: ",
           max_tokens=32,
           stop=["Q:", "\n"],
            echo = True,
)

In [10]:
resp['choices'][0]['text']

'Q: Selam nasılsın A: '

### Llama.cpp'de Chat Completion API Kullanımı

Llama.cpp'nin yüksek seviye Python API'si, sohbet tamamlama (chat completion) için basit bir arayüz sunar. Bu özellik, modelin birden fazla mesajı tek bir prompt'a nasıl formatlayacağını bilmesini gerektirir.

Llama sınıfı, mesajları tek bir prompt'a dönüştürürken şu sırayı izler:

1) chat_handler parametresi verilmişse bunu kullanır

2) chat_format parametresi verilmişse bunu kullanır

3) GGUF modelinin metadata'sındaki tokenizer.chat_template'i kullanır.
4) Hiçbiri yoksa llama-2 chat formatını kullanır
5) Çoklu-turn diyalog desteği sağlar

6) Sistem talimatlarını kullanıcı mesajlarından ayırır

In [15]:
from llama_cpp import Llama

llm = Llama(
    model_path="./models/gemma-3-12b-it-Q8_0.gguf", #model_path: GGUF formatındaki modelin dosya yolu
    chat_format="gemma" #chat_format: Kullanılacak sohbet formatı (örneğin "llama-2", "chatml", "gemma" vb.)
)
output = llm.create_chat_completion(
    messages=[ #messages: Sohbet mesajlarının listesi. 
        {"role":"system","content":"You are a perfect assistant"},
        {"role":"user","content":"Nasılsın ?"}
    ]
)

llama_model_loader: loaded meta data with 39 key-value pairs and 626 tensors from ./models/gemma-3-12b-it-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Gemma-3-12B-It
llama_model_loader: - kv   3:                           general.finetune str              = it
llama_model_loader: - kv   4:                           general.basename str              = Gemma-3-12B-It
llama_model_loader: - kv   5:                       general.quantized_by str              = Unsloth
llama_model_loader: - kv   6:                         general.size_label str              = 12B
llama_model_loader: - kv   7:                           g

load: control token: 261774 '<unused5872>' is not marked as EOG
load: control token: 261773 '<unused5871>' is not marked as EOG
load: control token: 261771 '<unused5869>' is not marked as EOG
load: control token: 261767 '<unused5865>' is not marked as EOG
load: control token: 261766 '<unused5864>' is not marked as EOG
load: control token: 261763 '<unused5861>' is not marked as EOG
load: control token: 261762 '<unused5860>' is not marked as EOG
load: control token: 261761 '<unused5859>' is not marked as EOG
load: control token: 261759 '<unused5857>' is not marked as EOG
load: control token: 261758 '<unused5856>' is not marked as EOG
load: control token: 261756 '<unused5854>' is not marked as EOG
load: control token: 261751 '<unused5849>' is not marked as EOG
load: control token: 261750 '<unused5848>' is not marked as EOG
load: control token: 261749 '<unused5847>' is not marked as EOG
load: control token: 261748 '<unused5846>' is not marked as EOG
load: control token: 261747 '<unused5845

In [24]:
output['choices'][0]["message"]['content']

'Ben bir dil modeliyim, bu yüzden duygularım yok. Ancak, bana sorduğun için teşekkür ederim! Şu anda iyiyim, senin nasıl olduğunu merak ediyorum?'

In [27]:
output = llm.create_chat_completion(
    messages=[ #messages: Sohbet mesajlarının listesi. 
        {"role":"system","content":"Harika bir asistansın"},
        {"role":"user","content":"merhaba?"}
    ]
)
output['choices'][0]["message"]['content']

Llama.generate: 4 prefix-match hit, remaining 8 prompt tokens to eval


llama_perf_context_print:        load time =   15802.51 ms
llama_perf_context_print: prompt eval time =   27227.16 ms /    16 tokens ( 1701.70 ms per token,     0.59 tokens per second)
llama_perf_context_print:        eval time =   10695.28 ms /    12 runs   (  891.27 ms per token,     1.12 tokens per second)
llama_perf_context_print:       total time =   17785.46 ms /    28 tokens


'Merhaba! Nasılsınız? Size nasıl yardımcı olabilirim?'

### Llama.cpp'de JSON ve JSON Schema Modları

Llama.cpp'nin JSON ve JSON Schema modları, model çıktılarını yapılandırılmış formatta almak için tasarlanmış güçlü özelliklerdir.



Cıktıları güzel. Chat olunca daha iyi performasn gösteriyor.

In [37]:
from llama_cpp import Llama

response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "Yanıtlarınızı JSON formatında verin."},
        {"role": "user", "content": "2020 Dünya Serisi'ni kim kazandı?"}
    ],
    response_format={"type": "json_object"},  # JSON modunu aktifleştirir
    temperature=0.7
)


Llama.generate: 21 prefix-match hit, remaining 1 prompt tokens to eval


llama_perf_context_print:        load time =   15802.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   11108.18 ms /    13 runs   (  854.48 ms per token,     1.17 tokens per second)
llama_perf_context_print:       total time =   11846.70 ms /    14 tokens


In [36]:
response['choices'][0]['message']['content']

'{\n"answer": "Los Angeles Dodgers"\n}'

In [38]:
response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "Yanıtlarınızı belirtilen JSON şemasına uygun verin."},
        {"role": "user", "content": "2020 Dünya Serisi'ni kim kazandı?"}
    ],
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "champion": {"type": "string"},
                "runner_up": {"type": "string"},
                "games_played": {"type": "integer"}
            },
            "required": ["champion", "runner_up"]
        }
    }
)

Llama.generate: 21 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   15802.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   19683.88 ms /    24 runs   (  820.16 ms per token,     1.22 tokens per second)
llama_perf_context_print:       total time =   21531.95 ms /    25 tokens


In [40]:
response['choices'][0]['message']['content']

'{ "champion": "Los Angeles Dodgers", "runner_up": "Tampa Bay Rays"}'

### Llama.cpp'de Fonksiyon Çağırma (Function Calling) Sistemi

Llama.cpp'nin fonksiyon çağırma özelliği, modelin dış işlevleri çağırabilmesini sağlayan güçlü bir mekanizmadır. Bu sistem, OpenAI'nin function calling API'siyle uyumlu şekilde çalışır.

- Temel Çalışma Prensibi
1) Model, kullanıcı sorgusunu analiz eder

2) Tanımlanan fonksiyonlardan uygun olanı seçer

3) Fonksiyon parametrelerini otomatik olarak doldurur

4) JSON formatında hazırlanmış fonksiyon çağrısını döndürür

In [55]:
from llama_cpp import Llama

llm = Llama(
    model_path="./models/gemma-3-12b-it-Q8_0.gguf",
    chat_format="chatml-function-calling", ## Fonksiyon çağırma desteği olan format.functionary: Özel eğitilmiş fonksiyon çağırma modeli
    verbose = False,
    #n_gpu_layers=40
   

)

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [45]:
tools = [{
    "type": "function",
    "function": {
        "name": "UserDetail",
        "description": "Kullanıcı bilgilerini çıkarır",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "age": {"type": "integer"}
            },
            "required": ["name", "age"]
        }
    }
}]

In [48]:
response = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "Gerekirse fonksiyonları çağırarak kullanıcıya yardımcı ol."
        },
        {
            "role": "user",
            "content": "ali 20 yaşında"
        }
    ],
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "UserDetail"}}
)

### Veri Çıkarımı (Information Extraction)

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "extract_contact_info",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "phone": {"type": "string"},
                "email": {"type": "string"}
            }
        }
    }
}]

### API Entegrasyonları

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
            },
            "required": ["location"]
        }
    }
}]

### Çoklu Fonksiyon Desteği için

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {...}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get stock price information",
            "parameters": {...}
        }
    }
]

In [58]:
import torch
print(f"GPU Aktif mi: {torch.cuda.is_available()}")
print(f"Kullanılan GPU Katmanları: {llm.model_params.n_gpu_layers}")

GPU Aktif mi: True
Kullanılan GPU Katmanları: 0


### Embeddings Nedir

Embeddings , metinleri sayısal vektörlere dönüştüren yapılardır. Bu vektörler
- Kelime veya cümleleri sabit uzunlukta sayı dizilerine çevirir

- Anlamsal ilişkileri yakalar (benzer anlamlar, benzer vektörler oluşturur)

- Metinler üzerinde matematiksel işlemlere izin verir (örneğin benzer ifadeleri bulma)

In [60]:
import llama_cpp

llm = llama_cpp.Llama(model_path="./models/gemma-3-12b-it-Q8_0.gguf", embedding=True)

embeddings = llm.create_embedding("Hello, world!")


embeddings = llm.create_embedding(["Hello, world!", "Goodbye, world!"])

llama_model_loader: loaded meta data with 39 key-value pairs and 626 tensors from ./models/gemma-3-12b-it-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Gemma-3-12B-It
llama_model_loader: - kv   3:                           general.finetune str              = it
llama_model_loader: - kv   4:                           general.basename str              = Gemma-3-12B-It
llama_model_loader: - kv   5:                       general.quantized_by str              = Unsloth
llama_model_loader: - kv   6:                         general.size_label str              = 12B
llama_model_loader: - kv   7:                           g

In [61]:
embeddings

{'object': 'list',
 'data': [{'object': 'embedding',
   'embedding': [[-0.12009039521217346,
     -0.03235245868563652,
     -0.5278037786483765,
     -0.26026424765586853,
     -0.019231274724006653,
     -0.12758991122245789,
     0.23753558099269867,
     0.19491612911224365,
     0.48707300424575806,
     0.35849907994270325,
     -0.37141159176826477,
     -0.017282634973526,
     -0.17354366183280945,
     0.05618826672434807,
     -0.16548338532447815,
     0.4246729612350464,
     0.04404093325138092,
     0.23986053466796875,
     0.2942980229854584,
     -0.43646666407585144,
     -0.08032135665416718,
     0.15812896192073822,
     0.19520536065101624,
     -0.09361202269792557,
     -0.051830485463142395,
     -0.1746188849210739,
     -0.09106790274381638,
     -0.05904803425073624,
     0.17201267182826996,
     0.044931210577487946,
     0.09291736781597137,
     -0.019817354157567024,
     0.32021796703338623,
     -0.04259900003671646,
     -0.06200640648603439,
     0